In [8]:
#import libraries


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from Bio import SeqIO
import filecmp
import subprocess
import os

%matplotlib inline

## Define functions

In [22]:
## Define functions
pattern1 = re.compile("GTCAG")
pattern_G = re.compile('[G]{2,}$')
adapt = "GTCAG"

def delete_Ns_before_adapter_fromR2(infile_path, outfile_path, pattern):
    '''
    Function removes the Ns sequence and pattern (adaptor) from the beginning of of second line of read in reads2, and move it to the header of read
    in the form:  @header_pattern.
    The function check length of the read and remove appropriate number of characters from fourth line of read
    (the line with the quality of read). 
    Reads without pattern have header with "_withoutanytail" at the end of the end of header.
    
    Input:
        infile_path: str
        outfile_path: str
        pattern: str
        
    Output: new fastq file (under output_path) withoth Ns sequence and the pattern, and truncated qualit line. 
    '''
    with open(outfile_path, 'w') as w:
        infile = SeqIO.parse(infile_path, "fastq")
        pattern = re.compile(pattern)

        for read in infile:
            fastq = read.format("fastq")
            header = fastq.partition('\n')[0]
            sequence = fastq.splitlines()[1]
            strand = fastq.splitlines()[2]
            quality = fastq.splitlines()[3]
            #pattern = re.compile(pattern)
            adapter = re.search(pattern, str(sequence))
         #   GG_seq_3prim = re.search(pattern_G, str(sequence))##
            if adapter == None:
                adapter1 = "lack_of_adapter"
            else:
                adapter = adapter.group(0)
               # GG_seq_3prim = GG_seq_3prim.group(0)##
                sequence = sequence.split(adapt, 1)[-1]
                #sequence = 
               # NNs = sequence.split(adapt)[0]
            
                seq_length = len(sequence)
                
                quality = str(quality[-(seq_length+1) :-1])
            #header=str(read.id) + "_" + NNs 
            new_read = header+'\n'+sequence+'\n'+strand+'\n'+ quality+'\n'
            w.write(new_read)
            
pattern1 = re.compile("GTCAG")
pattern_G = re.compile('[G]{2,}$')
adapt = "GTCAG"

def move_Ns_before_adapter_fromR2_to_header(infile_path, outfile_path, pattern):
    '''
    Function removes the Ns sequence and pattern (adaptor) from the beginning of of second line of read in reads2, and move it to the header of read
    in the form:  @header_pattern.
    The function check length of the read and remove appropriate number of characters from fourth line of read
    (the line with the quality of read). 
    Reads without pattern have header with "_withoutanytail" at the end of the end of header.
    
    Input:
        infile_path: str
        outfile_path: str
        pattern: str
        
    Output: new fastq file (under output_path) withoth Ns sequence and the pattern, and truncated qualit line. 
    '''
    with open(outfile_path, 'w') as w:
        infile = SeqIO.parse(infile_path, "fastq")
        pattern = re.compile(pattern)

        for read in infile:
            fastq = read.format("fastq")
            header = fastq.partition('\n')[0]
            sequence = fastq.splitlines()[1]
            strand = fastq.splitlines()[2]
            quality = fastq.splitlines()[3]
            #pattern = re.compile(pattern)
            adapter = re.search(pattern, str(sequence))
            seq_raw_length = len(sequence)
         #   GG_seq_3prim = re.search(pattern_G, str(sequence))##
            if adapter == None:
                adapter1 = "lack_of_adapter"
                sequence_NNs = "noNNs"
                header= '@'+str(read.id) + "_" + str(sequence_NNs)#
                new_read = header+'\n'+sequence+'\n'+strand+'\n'+ quality+'\n'
            else:
                adapter = adapter.group(0)
                sequence_NNs = sequence.split(adapt, 1)[0] 
            #GG_seq_3prim = GG_seq_3prim.group(0)##
                sequence = sequence.split(adapt, 1)[-1]
                
               # NNs = sequence.split(adapt)[0]
                seq_length = len(sequence)
                if seq_length != seq_raw_length:
                    quality = str(quality[-(seq_length+1) :-1])
                else:
                    quality = ''
                    sequence = ''
                header='@'+str(read.id) + "*5NN" + str(sequence_NNs)#
            new_read = header+'\n'+sequence+'\n'+strand+'\n'+ quality+'\n'
            w.write(new_read)
            
            
            
####### Function to extract tail from read sequence based on CIGAR #########
def take_tail_fromcigar2(strand, cig, seq):
    '''
    Function returns tail seqence based on the CIGAR and sequence of read
    Input:
        cig: CIGAR from sam file after alignment qith soft-clipping  [str]
        seq: sequence of read started from 3' end, having posibble tail   [str]
    Output:
        sequence of the tail extracted from read sequence based on CIGAR [str]
    Assumptions:
        1. For tailed reads, CIGAR starts from \d+S (softcliping code). I's about the number
        of softclipped nucleotides. We assume that this is also the length of the polyA tail.
        2. Sequence seq starts from 3' tail of read.
    '''

    cigar2 = str(cig) # convert cigar to string
    if not "S" in cigar2: # return nothing if CIGAR does not contain S (soft-clipping)
        return ""
    else:  # analyze data if CIGAR starts from number and S (3' soft-clipping). If not - return nothing
        
        
        if strand =='-':      
            search_result_minus = re.findall("[0-9]+S$", cigar2)
            if search_result_minus: # find CIGARs starting from 3' soft-clipping
                number_S = int(search_result_minus[-1].split('S')[0])
                # extract number of soft-clipped nucleotides
                tail = seq[-number_S:] # extract tail (based of number of soft-clipped nucleotides)
                return tail
            else:
                return ""
        elif strand == '+':
            search_result_plus = re.findall("^[0-9]+S", cigar2)
            if search_result_plus: # find CIGARs starting from 3' soft-clipping
                number_S = int(cigar2.split('S')[0]) # extract number of soft-clipped nucleotides
                tail = seq[0:number_S] # extract tail (based of number of soft-clipped nucleotides)
                return tail
            else:
                return ""
        else:
             return ""
        

# Test the function take_tail_fromcigar():

assert take_tail_fromcigar2('-', '8S', 'UUUHSAAAAAAAAAA') =='AAAAAAAA'
assert take_tail_fromcigar2('-', '8S444M1S', 'AAAAAAAAAAUUUHS') =='S'
assert take_tail_fromcigar2('+', '8S444M3S', 'AAAAAAAAAAUUUHS') =='AAAAAAAA'
assert take_tail_fromcigar2('-', '67M7S', 'AAAAAAAAAAUUUHS') =='AAUUUHS'
assert take_tail_fromcigar2('-', '*', 'AAAAAAAAAAUUUHS') ==''

def def_strand(flag):
    if flag == 4:
        return "unmapped"
    elif flag ==16:
        return "-"
    elif flag == 0:
        return "+"
    else:
        return 'nn'
    
    
    
    
pattern_mRNA = re.compile("spac|spap|spbp|spcc|spcp|spbc")
pattern_tRNA = re.compile("trna")
pattern_mito = re.compile("spmit")
pattern_rRNA = re.compile("rrna")
pattern_snRNA = re.compile("snrna")
pattern_snoRNA = re.compile("snorna")
pattern_ncRNA = re.compile("ncrna")


def test_RNA_type(feature_name):
    """tests for the type of tail """
    if pattern_mRNA.search(feature_name): # for example AATT
        return "mRNA"
    elif pattern_tRNA.search(feature_name): # for example AATT
        return "tRNA"
    elif pattern_mito.search(feature_name): # for example AATT
        return "mito_RNA"
    elif pattern_rRNA.search(feature_name): # for example AATT
        return "rRNA"
    elif pattern_snRNA.search(feature_name): # for example AATT
        return "snRNA"
    elif pattern_snoRNA.search(feature_name): # for example AATT
        return "snoRNA"
    elif pattern_ncRNA.search(feature_name): # for example AATT
        return "ncRNA"
    else: #only A's
        return "other_type"

assert test_RNA_type("spncrna") == "ncRNA"
assert test_RNA_type("sspcc") == "mRNA"
assert test_RNA_type("AAAAA") == "other_type"
assert test_RNA_type("TAATTT") == "other_type"

####### Function to test tail type #########

pattern_polyAU_forward = re.compile("^A{1,}T{2,}") # reverse transcribed
pattern_oligoU_forward = re.compile("^A{1,}")
pattern_polyA_forward = re.compile("^T{3,}$")

pattern_polyAU_reverse = re.compile("A{1,}T{2,}$") # reverse transcribed
pattern_oligoU_reverse = re.compile("T{1,}$")
pattern_polyA_reverse = re.compile("A{3,}$")  

def test_tail(strand, tail):
    """ Tests for the type of tail. Patterns are prepared for analysis of R2 read, reverse transcribed:
        base sequence start from the end of polyadenylation tail at 3' end of RNA,
        where T means adenine, and A meand uridine in the 3' tail"""
    if len(tail) > 0:
        if strand == '+' or strand == 'unmapped':
            if "A" in tail:
                if pattern_polyAU_forward.fullmatch(tail): 
                    return "polyAU"
                elif pattern_oligoU_forward.fullmatch(tail): 
                    return "oligoU"
                else:
                    return "mixed_tail"
            elif pattern_polyA_forward.fullmatch(tail):
                    return "polyA"
            else: 
                return "mixed_tail"
        elif strand == '-':
            if "T" in tail:
                if pattern_polyAU_reverse.fullmatch(tail): 
                    return "polyAU"
                elif pattern_oligoU_reverse.fullmatch(tail): 
                    return "oligoU"
                else:
                    return "mixed_tail"
            elif pattern_polyA_reverse.fullmatch(tail):
                    return "polyA"
            else: 
                return "mixed_tail"
    else:
        return "without_tail"
    
# Test the function test_tail():

assert test_tail('+', "AATTT") == "polyAU"
assert test_tail('+', "AAAAAA") == "oligoU"
assert test_tail('-',"AAAAA") == "polyA"
assert test_tail('-',"TAATTT") == "mixed_tail"
assert test_tail('+','')== "without_tail"
assert test_tail('unmapped','TTT')== "polyA"


def take_tail_fromcigar2(strand, cig, seq):
    '''
    Function returns tail seqence based on the CIGAR and sequence of read
    Input:
        cig: CIGAR from sam file after alignment qith soft-clipping  [str]
        seq: sequence of read started from 3' end, having posibble tail   [str]
    Output:
        sequence of the tail extracted from read sequence based on CIGAR [str]
    Assumptions:
        1. For tailed reads, CIGAR starts from \d+S (softcliping code). I's about the number
        of softclipped nucleotides. We assume that this is also the length of the polyA tail.
        2. Sequence seq starts from 3' tail of read.
    '''

    cigar2 = str(cig) # convert cigar to string
    if not "S" in cigar2: # return nothing if CIGAR does not contain S (soft-clipping)
        return ""
    else:  # analyze data if CIGAR starts from number and S (3' soft-clipping). If not - return nothing
        
        
        if strand =='-':      
            search_result_minus = re.findall("[0-9]+S$", cigar2)
            if search_result_minus: # find CIGARs starting from 3' soft-clipping
                number_S = int(search_result_minus[-1].split('S')[0])
                # extract number of soft-clipped nucleotides
                tail = seq[-number_S:] # extract tail (based of number of soft-clipped nucleotides)
                return tail
            else:
                return ""
        elif strand == '+':
            search_result_plus = re.findall("^[0-9]+S", cigar2)
            if search_result_plus: # find CIGARs starting from 3' soft-clipping
                number_S = int(cigar2.split('S')[0]) # extract number of soft-clipped nucleotides
                tail = seq[0:number_S] # extract tail (based of number of soft-clipped nucleotides)
                return tail
            else:
                return ""
        else:
             return ""
        

# Test the function take_tail_fromcigar():

assert take_tail_fromcigar2('-', '8S', 'UUUHSAAAAAAAAAA') =='AAAAAAAA'
assert take_tail_fromcigar2('-', '8S444M1S', 'AAAAAAAAAAUUUHS') =='S'
assert take_tail_fromcigar2('+', '8S444M3S', 'AAAAAAAAAAUUUHS') =='AAAAAAAA'
assert take_tail_fromcigar2('-', '67M7S', 'AAAAAAAAAAUUUHS') =='AAUUUHS'
assert take_tail_fromcigar2('-', '*', 'AAAAAAAAAAUUUHS') ==''


def grep_tail_edit(strand, seq):
    '''
    Function ....
    '''

    if strand =='-':      
        search_result_minus = re.findall("[A]{0,}[T]{0,}$", seq)
        if search_result_minus: # find CIGARs starting from 3' soft-clipping
            return search_result_minus[0]
        else:
            return ""
    elif strand == '+' or strand == 'unmapped':
        search_result_plus = re.findall("^[A]{0,}[T]{0,}", seq)
        if search_result_plus: # find CIGARs starting from 3' soft-clipping
            return search_result_plus[0]
        else:
            return ""
    else:
        return ""
        

# Test the function take_tail_fromcigar():

assert grep_tail_edit('-', 'UUUHSAAAAAAAAAA') =='AAAAAAAAAA'
assert grep_tail_edit('-',  'AAAAAAAAAAUUUHS') ==''
assert grep_tail_edit('+',  'AAAAAAAAAAUUUHS') =='AAAAAAAAAA'

def tail_fromGREPorCIGAR_description(cigar, tailGrep, tailCigar):
    if cigar == '*':
        return 'grep'
    else:
        return 'cigar'


def tail_fromGREPorCIGAR(cigar, tailGrep, tailCigar):
    if cigar == '*':
        return tailGrep
    else:
        return tailCigar

In [ ]:
# Analysis of Tail-seq data using both reads, R1 and R2



## Selection of R2 reads having UMIs and adapter

In [3]:
!ls data/*_R2_001.fastq.gz

data/deltacid1_clone1_S77_R2_001.fastq.gz
data/deltacid1_clone2_S78_R2_001.fastq.gz
data/deltacid1deltadis32_clone1_S85_R2_001.fastq.gz
data/deltacid1deltadis32_clone2_S86_R2_001.fastq.gz
data/deltacid1deltadis32deltalsm1_clone1_S97_R2_001.fastq.gz
data/deltacid1deltadis32deltalsm1_clone2_S98_R2_001.fastq.gz
data/deltacid1deltadis32deltaski2_clone1_S99_R2_001.fastq.gz
data/deltacid1deltadis32deltaski2_clone2_S100_R2_001.fastq.gz
data/deltacid1deltalsm1_clone1_S87_R2_001.fastq.gz
data/deltacid1deltalsm1_clone2_S88_R2_001.fastq.gz
data/deltacid1deltalsm1deltaski2_clone1_S101_R2_001.fastq.gz
data/deltacid1deltalsm1deltaski2_clone2_S102_R2_001.fastq.gz
data/deltacid1deltaski2_clone1_S89_R2_001.fastq.gz
data/deltacid1deltaski2_clone2_S90_R2_001.fastq.gz
data/deltadis32_clone1_S79_R2_001.fastq.gz
data/deltadis32_clone2_S80_R2_001.fastq.gz
data/deltadis32deltalsm1_clone1_S91_R2_001.fastq.gz
data/deltadis32deltalsm1_clone2_S92_R2_001.fastq.gz
data/deltadis32deltalsm1deltaski2_clone1_S103_R2_00

In [4]:
!mkdir R2_allsamples
!mkdir R2_allsamples/filter

In [6]:
## NEWscript1_selectR2withUNIs.bash


#!/bin/bash

# Declare an array of string with type
declare -a StringArray=('deltacid1_clone1_S77_' 'deltacid1_clone2_S78_' 'deltacid1deltadis32_clone1_S85_' 'deltacid1deltadis32_clone2_S86_' 'deltacid1deltadis32deltalsm1_clone1_S97_' 'deltacid1deltadis32deltalsm1_clone2_S98_' 'deltacid1deltadis32deltaski2_clone1_S99_' 'deltacid1deltadis32deltaski2_clone2_S100_' 'deltacid1deltalsm1_clone1_S87_' 'deltacid1deltalsm1_clone2_S88_' 'deltacid1deltalsm1deltaski2_clone1_S101_' 'deltacid1deltalsm1deltaski2_clone2_S102_' 'deltacid1deltaski2_clone1_S89_' 'deltacid1deltaski2_clone2_S90_' 'deltadis32_clone1_S79_' 'deltadis32_clone2_S80_' 'deltadis32deltalsm1_clone1_S91_' 'deltadis32deltalsm1_clone2_S92_' 'deltadis32deltalsm1deltaski2_clone1_S103_' 'deltadis32deltalsm1deltaski2_clone2_S104_' 'deltadis32deltaski2_clone1_S93_' 'deltadis32deltaski2_clone2_S94_' 'deltalsm1_clone1_S81_' 'deltalsm1_clone2_S82_' 'deltalsm1deltaski2_clone1_S95_' 'deltalsm1deltaski2_clone2_S96_' 'deltaski2_clone1_S83_' 'deltaski2_clone2_S84_' 'Wild_type_clone1_TS_S75_' 'Wild_type_clone2_TS_S76_')

# Iterate the string array using for loop
# Gunzip R2.fastq, select reads having 5Ns and adaptor, gzip input fastq
for val in ${StringArray[@]}; do 
	echo "sample_${val}"; gunzip "data/${val}R2_001.fastq.gz"; grep -E -A 2 -B 1 --no-group-separator '^[A-Z][A-Z][A-Z][A-Z][A-Z][A-Z]GTCAG' "data/${val}R2_001.fastq" > "R2_allsamples/filter/${val}R2_withUMIs.fastq"; gzip "data/${val}R2_001.fastq"; 
done

/bin/sh: 1: Syntax error: "(" unexpected
/bin/sh: 1: Bad substitution


In [ ]:
#gunzip data/Wild_type_clone1_TS_S75_R2_001.fastq.gz
#grep -E -A 2 -B 1 --no-group-separator '^[A-Z][A-Z][A-Z][A-Z][A-Z][A-Z]GTCAG' data/Wild_type_clone1_TS_S75_R2_001.fastq > analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq
#gzip data/Wild_type_clone1_TS_S75_R2_001.fastq



In [20]:
!ls

 all_mapped_po1nt+kopia.fastq
 analiza_July2022
'Analiza short.ipynb'
 analiza_unmapped_tailed_loop
 background_angeli_554genes_20220311.txt
 bams
 bams_STAR
 BED_R2_STAR
 bigwigs
 data
 deltacid1_clone1_S77_tailsinheader_20220119.fastq
 DESEQ_LL.R
 empty.fastq
 figures
 gene_cluster_20220309.csv
 genome
 genome_sdsintronsutrs
 gff
 htseq
 htseq_merged3.csv
 htseq_mRNA_doDESeq2_20220113.csv
 image.webp
 kopia_all_mapped_po1nt.fastq
 lista_headerow.txt
 Log.out
 mRNA_tailtypes_per_strain_20220221.csv
 multiqc
 NEWscript1_selectR2withUMIs.bash
 output_headertoorganise.fastq
 pattern_in_header.fastq
 polyA_pivot_dropna_20220311.csv
 PyPolyADetector.py
 R1
 R1b.bed
 R1.bed
 R1b.sam
 R1.sam
 R1_uniqlymappedreads_20220125.csv
 R1_uniqlymappedreads_procent_20220125.csv
 R2_allsamples
 R2b.bed
 R2b.sam
 R2_R1_merge_sam4_PAS.csv
 R2_R1_wt_test_20220712.bed
 R2_R1_wt_test.bed
 R2_R2_sam3_test_20220712.csv
 R2_R2_sam3_test_20220713.csv
 R2_R2_sam3_test_20220714.csv
 R2_R2_sam_test_20220712.csv
 R

In [19]:
for i in range(0,10):
    os.system('ls')

In [23]:
#remove the primer GTCAG and move UMIs to the header of reads (Python script),
#
#move_Ns_before_adapter_fromR2_to_header(infile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq', outfile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_UMIinhHeader.fastq', pattern = 'GTCAG')
#usuwanie umi i NNs
names = ['deltacid1_clone1_S77_',  'deltacid1_clone2_S78_',  'deltacid1deltadis32_clone1_S85_',  
         'deltacid1deltadis32_clone2_S86_',  'deltacid1deltadis32deltalsm1_clone1_S97_',  
         'deltacid1deltadis32deltalsm1_clone2_S98_',  'deltacid1deltadis32deltaski2_clone1_S99_',  
         'deltacid1deltadis32deltaski2_clone2_S100_',  'deltacid1deltalsm1_clone1_S87_',  
         'deltacid1deltalsm1_clone2_S88_',  'deltacid1deltalsm1deltaski2_clone1_S101_',  
         'deltacid1deltalsm1deltaski2_clone2_S102_',  'deltacid1deltaski2_clone1_S89_',  
         'deltacid1deltaski2_clone2_S90_',  'deltadis32_clone1_S79_',  'deltadis32_clone2_S80_',  
         'deltadis32deltalsm1_clone1_S91_',  'deltadis32deltalsm1_clone2_S92_',  
         'deltadis32deltalsm1deltaski2_clone1_S103_',  'deltadis32deltalsm1deltaski2_clone2_S104_',  
         'deltadis32deltaski2_clone1_S93_',  'deltadis32deltaski2_clone2_S94_',  'deltalsm1_clone1_S81_',  
         'deltalsm1_clone2_S82_',  'deltalsm1deltaski2_clone1_S95_',  'deltalsm1deltaski2_clone2_S96_', 
         'deltaski2_clone1_S83_',  'deltaski2_clone2_S84_',  'Wild_type_clone1_TS_S75_', 
         'Wild_type_clone2_TS_S76_']

for name in names:
    print(name)
    delete_Ns_before_adapter_fromR2(infile_path = 'R2_allsamples/filter/{}R2_withUMIs.fastq'.format(name),   outfile_path = 'R2_allsamples/filter/{}R2_removedUMIs.fastq'.format(name),    
                                    pattern = 'GTCAG')
   # os.system('rm R2_allsamples/filter/{}R2_withUMIs.fastq'.format(name))


In [ ]:
!rm R2_allsamples/filter/*_withUMIs.fastq

## Selection of R1 reads based on R2 reads having adaptors - fast-pair

In [29]:
###########TEST##########################

## NEWscript2_selectR1fromR2_fastpair.bash


#!/bin/bash

# Declare an array of string with type
#declare -a StringArray=('deltacid1_clone1_S77_' 'deltacid1_clone2_S78_' 'deltacid1deltadis32_clone1_S85_' 'deltacid1deltadis32_clone2_S86_' 'deltacid1deltadis32deltalsm1_clone1_S97_' 'deltacid1deltadis32deltalsm1_clone2_S98_' 'deltacid1deltadis32deltaski2_clone1_S99_' 'deltacid1deltadis32deltaski2_clone2_S100_' 'deltacid1deltalsm1_clone1_S87_' 'deltacid1deltalsm1_clone2_S88_' 'deltacid1deltalsm1deltaski2_clone1_S101_' 'deltacid1deltalsm1deltaski2_clone2_S102_' 'deltacid1deltaski2_clone1_S89_' 'deltacid1deltaski2_clone2_S90_' 'deltadis32_clone1_S79_' 'deltadis32_clone2_S80_' 'deltadis32deltalsm1_clone1_S91_' 'deltadis32deltalsm1_clone2_S92_' 'deltadis32deltalsm1deltaski2_clone1_S103_' 'deltadis32deltalsm1deltaski2_clone2_S104_' 'deltadis32deltaski2_clone1_S93_' 'deltadis32deltaski2_clone2_S94_' 'deltalsm1_clone1_S81_' 'deltalsm1_clone2_S82_' 'deltalsm1deltaski2_clone1_S95_' 'deltalsm1deltaski2_clone2_S96_' 'deltaski2_clone1_S83_' 'deltaski2_clone2_S84_' 'Wild_type_clone1_TS_S75_' 'Wild_type_clone2_TS_S76_')

# Iterate the string array using for loop
# Gunzip R2.fastq, select reads having 5Ns and adaptor, gzip input fastq
!for val in 'deltacid1_clone1_S77_'; do rm "R2_allsamples/filter/${val}R2_removedUMIs.fastq" "R2_allsamples/filter/${val}R2_allsamples/filter/${val}R2_removedUMIs.fastq.paired.fq" "R2_allsamples/filter/${val}R2_removedUMIs.fastq.single.fq"; done

rm: cannot remove 'R2_allsamples/filter/deltacid1_clone1_S77_R2_allsamples/filter/deltacid1_clone1_S77_R2_removedUMIs.fastq.paired.fq': No such file or directory
rm: cannot remove 'R2_allsamples/filter/deltacid1_clone1_S77_R2_removedUMIs.fastq.single.fq': No such file or directory


In [ ]:
## NEWscript2_selectR1fromR2_fastpair.bash


#!/bin/bash

# Declare an array of string with type
declare -a StringArray=('deltacid1_clone1_S77_' 'deltacid1_clone2_S78_' 'deltacid1deltadis32_clone1_S85_' 'deltacid1deltadis32_clone2_S86_' 'deltacid1deltadis32deltalsm1_clone1_S97_' 'deltacid1deltadis32deltalsm1_clone2_S98_' 'deltacid1deltadis32deltaski2_clone1_S99_' 'deltacid1deltadis32deltaski2_clone2_S100_' 'deltacid1deltalsm1_clone1_S87_' 'deltacid1deltalsm1_clone2_S88_' 'deltacid1deltalsm1deltaski2_clone1_S101_' 'deltacid1deltalsm1deltaski2_clone2_S102_' 'deltacid1deltaski2_clone1_S89_' 'deltacid1deltaski2_clone2_S90_' 'deltadis32_clone1_S79_' 'deltadis32_clone2_S80_' 'deltadis32deltalsm1_clone1_S91_' 'deltadis32deltalsm1_clone2_S92_' 'deltadis32deltalsm1deltaski2_clone1_S103_' 'deltadis32deltalsm1deltaski2_clone2_S104_' 'deltadis32deltaski2_clone1_S93_' 'deltadis32deltaski2_clone2_S94_' 'deltalsm1_clone1_S81_' 'deltalsm1_clone2_S82_' 'deltalsm1deltaski2_clone1_S95_' 'deltalsm1deltaski2_clone2_S96_' 'deltaski2_clone1_S83_' 'deltaski2_clone2_S84_' 'Wild_type_clone1_TS_S75_' 'Wild_type_clone2_TS_S76_')

# Iterate the string array using for loop
# Gunzip R2.fastq, select reads having 5Ns and adaptor, gzip input fastq
for val in ${StringArray[@]}; do 
	echo "sample_${val}"; gunzip "data/${val}R1_001.fastq.gz"; fastq_pair "data/${val}R1_001.fastq" "R2_allsamples/filter/${val}R2_removedUMIs.fastq"; gzip "data/${val}R1_001.fastq"; rm "data/${val}R1_001.fastq.single.fq" "R2_allsamples/filter/${val}R2_removedUMIs.fastq" "R2_allsamples/filter/${val}R2_allsamples/filter/${val}R2_removedUMIs.fastq.paired.fq" "R2_allsamples/filter/${val}R2_removedUMIs.fastq.single.fq"; 
done

In [31]:
!gzip data/*R1_001.fastq.*.fq

In [ ]:
##cp data/Wild_type_clone1_TS_S75_R1_001.fastq.gz analiza_July2022/R1/
#gunzip  analiza_July2022/R1/Wild_type_clone1_TS_S75_R1_001.fastq.gz#
#fastq_pair analiza_July2022/R1/Wild_type_clone1_TS_S75_R1_001.fastq analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq
#Interesuje nas teraz plik: analiza_July2022/R1/Wild_type_clone1_TS_S75_R1_001.fastq.paired.fq


## Alignment of R1 to transcriptome using salmon


In [ ]:
!conda activate salmon

In [ ]:
!mkdir R2_allsamples/alignmentR1

In [ ]:
# prepare index 
salmon index -t salmon/cds+introns+utrs.fa.gz -i salmon/pombe_index

In [ ]:
# alignment

#salmon quant -i salmon/pombe_index -l MSF -r analiza_July2022/R1/Wild_type_clone1_TS_S75_R1_001.fastq.paired_cdpt2_min15.fastq.gz   -p 8 --validateMappings -o salmon/quants/Wild_type_clone1_TS_S75_quant  --writeMappings salmon/quants/Wild_type_clone1_TS_S75.sam


In [ ]:
## NEWscript3_alignmentR1_salmon.bash

#!/bin/bash

# Declare an array of string with type
declare -a StringArray=('deltacid1_clone1_S77_' 'deltacid1_clone2_S78_' 'deltacid1deltadis32_clone1_S85_' 'deltacid1deltadis32_clone2_S86_' 'deltacid1deltadis32deltalsm1_clone1_S97_' 'deltacid1deltadis32deltalsm1_clone2_S98_' 'deltacid1deltadis32deltaski2_clone1_S99_' 'deltacid1deltadis32deltaski2_clone2_S100_' 'deltacid1deltalsm1_clone1_S87_' 'deltacid1deltalsm1_clone2_S88_' 'deltacid1deltalsm1deltaski2_clone1_S101_' 'deltacid1deltalsm1deltaski2_clone2_S102_' 'deltacid1deltaski2_clone1_S89_' 'deltacid1deltaski2_clone2_S90_' 'deltadis32_clone1_S79_' 'deltadis32_clone2_S80_' 'deltadis32deltalsm1_clone1_S91_' 'deltadis32deltalsm1_clone2_S92_' 'deltadis32deltalsm1deltaski2_clone1_S103_' 'deltadis32deltalsm1deltaski2_clone2_S104_' 'deltadis32deltaski2_clone1_S93_' 'deltadis32deltaski2_clone2_S94_' 'deltalsm1_clone1_S81_' 'deltalsm1_clone2_S82_' 'deltalsm1deltaski2_clone1_S95_' 'deltalsm1deltaski2_clone2_S96_' 'deltaski2_clone1_S83_' 'deltaski2_clone2_S84_' 'Wild_type_clone1_TS_S75_' 'Wild_type_clone2_TS_S76_')

# Iterate the string array using for loop
# Align R1 using salmon. Generate counttable and sam file as output.
for val in ${StringArray[@]}; do 
	echo "sample_${val}"; salmon quant -i salmon/pombe_index -l MSF -r  "data/${val}_R1_001.fastq.paired.fq.gz" -p 8 --validateMappings -o "R2_allsamples/alignmentR1/${val}R1_quant"  --writeMappings "R2_allsamples/alignmentR1/${val}R1_salmon.sam"; 
done


In [ ]:
# selection of mapped R1 reads, and converting sam to bam, and then to fastq
samtools view -h -o salmon/quants/Wild_type_clone1_TS_S75.bam salmon/quants/Wild_type_clone1_TS_S75.sam
samtools view -b -F 4 salmon/quants/Wild_type_clone1_TS_S75.bam > salmon/quants/Wild_type_clone1_TS_S75_mapped.bam

## Selection of reads R2, which mapped to R1 with salmon

In [ ]:
fastq_pair salmon/quants/Wild_type_clone1_TS_S75_mapped.fastq analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq
#important output: analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq.paired.fq


## Alignment of selected R2 with STAR

In [ ]:
# preparation of data 
 move_Ns_before_adapter_fromR2_to_header(infile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq.paired.fq',
                                         outfile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_movedUMIS.fq', pattern = 'GTCAG')


In [ ]:
#alignment
STAR --genomeDir genome/ --readFilesIn analiza_July2022/R2/Wild_type_clone1_R2_movedUMIS.fq --alignIntronMax 1000 --outSAMtype BAM Unsorted --outFileNamePrefix analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_ –outFilterMultimapNmax 1

## Selekcja odczytów R2, które mają zmapowane odpowiedniki w R1

In [ ]:
 move_Ns_before_adapter_fromR2_to_header(infile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_GTCAG.fastq.paired.fq',
                                         outfile_path = 'analiza_July2022/R2/Wild_type_clone1_R2_movedUMIS.fq', pattern = 'GTCAG')

## Alignment of R2 reads

In [ ]:
!STAR --genomeDir genome/ --readFilesIn analiza_July2022/R2/Wild_type_clone1_R2_movedUMIS.fq --alignIntronMax 1000 --outSAMtype BAM Unsorted --outFileNamePrefix analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_ –outFilterMultimapNmax 1 --outSAMunmapped Within

## Down stream data analysis

## Import libraries

In [ ]:
!samtools view -h -o analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_Aligned.out.sam analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_Aligned.out.bam

#sam to bed
!sam2bed < analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_Aligned.out.sam > analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_Aligned.out.bed


In [ ]:
## wersja z bed zamiast z sam
!awk -F"\t" '{print $1,$2,$3,$4,$6,$8,$9,$12}' analiza_July2022/R2/Wild_type_clone1_R2_tylkozmapzR1_UMIinheader_Aligned.out.bed > R2b.bed

In [ ]:
r2_sam = pd.read_csv('R2b.sam', sep = ' ',names=['readID', 'flag', 'chr','coord', 'num', 'cigar', 'seq'])
r2_sam = r2_sam[(r2_sam['flag']<200)]
r2_sam['strand'] = r2_sam['flag'].apply(lambda x: def_strand(x))
r2_sam['UMIs'] = r2_sam['readID'].apply(lambda x: x.split('*5NN')[-1])
r2_sam['UMIs_seq'] = r2_sam['UMIs'] + '_' + r2_sam['seq']

r2_sam['readID'] = r2_sam['readID'].apply(lambda x: x.split('*5NN')[0])
r2_sam  = r2_sam.drop(columns = ['flag'])
#r2_sam.drop_duplicates()
r2_sam.head()

In [ ]:
r2_bed = pd.read_csv('R2b.bed', sep = ' ',names=['chr','start', 'stop','readID', 'strand','cigar','*', 'seq'])
r2_bed['start_read'] = r2_bed['start']
r2_bed['stop_read'] = r2_bed['stop']
r2_bed['readID'] = r2_bed['readID'].apply(lambda x: x.split('*5NN')[0])
r2_bed = r2_bed.drop(columns=['chr', 'start', 'stop','strand', 'cigar', '*', 'seq'])
r2_bed = r2_bed.drop_duplicates()

In [ ]:
## R1
!grep A00805 salmon/quants/Wild_type_clone1_TS_S75.sam > R1.sam
!sam2bed < R1.sam > R1.bed
!awk -F"\t" '{print $1,$4,$6,$7}' R1.bed > R1b.bed
!awk -F"\t" '{print $1,$2,$3}' R1.sam > R1b.sam

In [ ]:
r1_bed = pd.read_csv('R1b.bed', sep = ' ',names=['gene','readID','strand','flag_R1'])
r1_bed = r1_bed[r1_bed['flag_R1']<2000]
r1_bed.drop_duplicates()


In [ ]:
!head reference/newSP.bed
reference_newSP = pd.read_csv('reference/newSP.bed', names=['chr', 'start', 'stop', 'gene','1','2','3',
                                                           '4', '5', '6'], sep = '\t')

reference_newSP = reference_newSP.drop(columns=['1','2','3','4', '5', '6'])
reference_newSP.head()

In [ ]:
r1_sam = pd.read_csv('R1b.sam', sep = ' ',names=['readID','flag_R1', 'gene_R1'])
r1_sam_bezzbednychflag = r1_sam[r1_sam['flag_R1']<2000]

In [ ]:
r3 = r2_sam.drop(columns=[  'num'])
r3 = r3.drop_duplicates()

In [ ]:
R2_R1_merge_sam = pd.merge(r3, r1_bed, on = ['readID'])
#R2_R1_merge_sam2 = pd.merge(R2_R1_merge_sam, r2_bed, on = ['readID'])
R2_R1_merge_sam2 = R2_R1_merge_sam.drop_duplicates()
R2_R1_merge_sam2.head()

In [ ]:
R2_R1_merge_sam2['RNA_type'] = R2_R1_merge_sam2['gene'].apply(lambda x: test_RNA_type(x.lower()))

In [ ]:
R2_R1_merge_sam2['tail_fromcigar'] = R2_R1_merge_sam2.apply(lambda kol: take_tail_fromcigar2(kol.strand_y, kol.cigar,kol.seq), axis = 1)
R2_R1_merge_sam2['tail_fromGREP'] = R2_R1_merge_sam2.apply(lambda kol: grep_tail_edit(kol.strand_y,  kol.seq), axis = 1)
R2_R1_merge_sam2['tail_type_CIGAR'] = R2_R1_merge_sam2.apply(lambda kol: test_tail(kol.strand_y, kol.tail_fromcigar), axis = 1)
R2_R1_merge_sam2['tail_len_CIGAR'] = R2_R1_merge_sam2['tail_fromcigar'].apply(lambda x: len(x))
R2_R1_merge_sam2['tail_type_GREP'] = R2_R1_merge_sam2.apply(lambda kol: test_tail(kol.strand_y, kol.tail_fromGREP), axis = 1)
R2_R1_merge_sam2['tail_len_GREP'] = R2_R1_merge_sam2['tail_fromGREP'].apply(lambda x: len(x))

In [ ]:
R2_R1_merge_sam2 = R2_R1_merge_sam2.drop(columns=['cigar'])
R2_R1_merge_sam2 = R2_R1_merge_sam2.drop_duplicates()

In [ ]:
R2_R1_merge_sam2['tail_GreporCigar'] = R2_R1_merge_sam2.apply(lambda kol: tail_fromGREPorCIGAR(kol.cigar, kol.tail_fromGREP,kol.tail_fromcigar), axis = 1)
R2_R1_merge_sam2['tail_from'] = R2_R1_merge_sam2.apply(lambda kol: tail_fromGREPorCIGAR_description(kol.cigar, kol.tail_fromGREP,kol.tail_fromcigar), axis = 1)
R2_R1_merge_sam3 = pd.merge(R2_R1_merge_sam2,reference_newSP,on = ['gene'], how='left')

In [ ]:
..._to_csv('...')

In [ ]:
R2_R1_merge_sam3_in = R2_R1_merge_sam2.set_index('readID')
r2_bed_in = r2_bed.set_index('readID')

R2_R1_merge_sam4 = R2_R1_merge_sam3_in.join(r2_bed_in)

In [ ]:
closest_seq_plus = R2_R1_merge_sam4[R2_R1_merge_sam4['strand_y'] == '+']
closest_seq_minus = R2_R1_merge_sam4[R2_R1_merge_sam4['strand_y'] == '-']

closest_seq_plus['distance'] = -(closest_seq_plus['stop'] - closest_seq_plus['stop_read'])
closest_seq_minus['distance'] = -(closest_seq_minus['start_read'] - closest_seq_minus['start'])

R2_R1_merge_sam4_PAS = pd.concat([closest_seq_minus, closest_seq_plus])